## Laboratorio 2

###### Maria Isabel Ortiz Naranjo
###### Luis Alejandro Urbina
###### Catedratico: Jorge Yass 
###### Catedra: Security Data Science

In [2]:
import pandas as pd
import numpy as np
import nltk
from sklearn.feature_extraction.text import CountVectorizer
import contractions
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /home/isa/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/isa/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/isa/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/isa/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

## Exploracion de datos 

In [4]:
df = pd.read_csv("enronSpamSubset.csv")
df1 = pd.read_csv("completeSpamAssassin.csv")

In [5]:
df.head()

,Unnamed: 0,Unnamed: 0.1,Body,Label
0,2469,2469,Subject: stock promo mover : cwtd\n * * * urge...,1
1,5063,5063,Subject: are you listed in major search engine...,1
2,12564,12564,"Subject: important information thu , 30 jun 20...",1
3,2796,2796,Subject: = ? utf - 8 ? q ? bask your life with...,1
4,1468,1468,"Subject: "" bidstogo "" is places to go , things...",1


In [8]:
df1.head()

,Unnamed: 0,Body,Label
0,0,\nSave up to 70% on Life Insurance.\nWhy Spend...,1
1,1,1) Fight The Risk of Cancer!\nhttp://www.adcli...,1
2,2,1) Fight The Risk of Cancer!\nhttp://www.adcli...,1
3,3,##############################################...,1
4,4,I thought you might like these:\n1) Slim Down ...,1


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    10000 non-null  int64 
 1   Unnamed: 0.1  10000 non-null  int64 
 2   Body          10000 non-null  object
 3   Label         10000 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 312.6+ KB


In [10]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6046 entries, 0 to 6045
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  6046 non-null   int64 
 1   Body        6045 non-null   object
 2   Label       6046 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 141.8+ KB


In [11]:
thw = [df,df1]

In [12]:
df_or = pd.concat(thw)

In [13]:
df_or.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16046 entries, 0 to 6045
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    16046 non-null  int64  
 1   Unnamed: 0.1  10000 non-null  float64
 2   Body          16045 non-null  object 
 3   Label         16046 non-null  int64  
dtypes: float64(1), int64(2), object(1)
memory usage: 626.8+ KB


In [14]:
df_or.head()

,Unnamed: 0,Unnamed: 0.1,Body,Label
0,2469,2469.0,Subject: stock promo mover : cwtd\n * * * urge...,1
1,5063,5063.0,Subject: are you listed in major search engine...,1
2,12564,12564.0,"Subject: important information thu , 30 jun 20...",1
3,2796,2796.0,Subject: = ? utf - 8 ? q ? bask your life with...,1
4,1468,1468.0,"Subject: "" bidstogo "" is places to go , things...",1


In [16]:
df_or = df_or[['Body', 'Label']]
df_or.head()

,Body,Label
0,Subject: stock promo mover : cwtd\n * * * urge...,1
1,Subject: are you listed in major search engine...,1
2,"Subject: important information thu , 30 jun 20...",1
3,Subject: = ? utf - 8 ? q ? bask your life with...,1
4,"Subject: "" bidstogo "" is places to go , things...",1


## Preprocesamiento

###### Codigo inspirado por el notebook visto en clase: Preprocesamiento.ipynb

In [23]:
import unicodedata
import re

df_or['Body'] = df_or['Body'].str.strip()
df_or['Body'] = df_or['Body'].str.lower()

def removerAcentos(texto):
    texto = unicodedata.normalize('NFKD', str(texto)).encode('ascii','ignore').decode('utf-8','ignore')
    return texto

def removerCaracteresEspecialesNumerosSimbolos(texto, removerDigitos =False):
    patron = r'[^a-zA-Z0-9\s]' if not removerDigitos else r'[^a-zA-Z\s]'
    texto = re.sub(patron,'', texto, re.I|re.A)
    return texto

In [26]:
df_or['Body'] = df_or['Body'].apply(removerAcentos)
df_or['Body'] = df_or['Body'].apply(removerCaracteresEspecialesNumerosSimbolos)
df_or['Body'] = df_or['Body'].apply(contractions.fix)
stop_words_english = nltk.corpus.stopwords.words('english')

In [30]:
def doc(texto):
    tokens = nltk.word_tokenize(texto)
    filtered_tokens = [token for token in tokens if token not in stop_words_english]
    texto = ' '.join(filtered_tokens)
    return texto

df_or['Body'] = df_or['Body'].apply(doc)
df_or.head


<bound method NDFrame.head of                                                    Body  Label
0     subject stock promo mover cwtd urgent investor...      1
1     subject listed major search engines submitting...      1
2     subject important information thu 30 jun 2005 ...      1
3     subject utf 8 q bask life utf 8 q individual i...      1
4     subject bidstogo places go things hello privac...      1
...                                                 ...    ...
6041                                              empty      0
6042  oneten 271102 insignificant matters heavily ov...      0
6043  issue01 readers write 02 extension search 03 d...      0
6044                                              empty      0
6045                                              empty      0

[16046 rows x 2 columns]>